<a href="https://colab.research.google.com/github/drewscottt/UArizona-RMP/blob/main/UArizona_RMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is my final project for ISTA322. In it, I extract information about professors at the University of Arizona from two sources (a salary sheet and RateMyProfessor API). Then, I transform the data to filter interesting information about the data, use libraries to infer demographic information about the professors, and aggregate data to create datasets for each department and course at the UofA. Then, I load all of this information into a database. I have example queries for each of the tables loaded into the DB, showing what sorts of questions can be asked about the data.

[My original plan](https://docs.google.com/document/d/1RWUIrtQrgzBvJRRMYXLDIr0429hGngkLyFeovoPUHlE/edit?usp=sharing)

##Installations
This will automatically restart the runtime environment (otherwise errors will be caused later on in the project), so a crashed message will show, but that's okay.

In [ ]:
!pip install -U -q jsonmerge
!pip install gender-guesser
!pip install ethnicolr
!pip install mysql-connector-python

# need to restart runtime for some reason
import os
os.kill(os.getpid(), 9)

##Import modules

In [ ]:
# used to read from Google Drive 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# used to request from APIs and manage their json responses
import urllib.request
import json
#from jsonmerge import merge, Merger

import mysql.connector as mysql

import pandas as pd
import numpy as np
import math

# used to infer demographic information about professors
import gender_guesser.detector as gender
from ethnicolr import census_ln

##Google Drive Authentication


In [ ]:
# Authenticate access to Google Drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Extract
Data sources:
* [UArizona Employee Salaries (2019-20)](https://docs.google.com/spreadsheets/d/e/2PACX-1vTaAWak0pN6Jnulm95eTM7kIubvNNMPgYh3d6sCHN5W1tekpIktoMBoDKJeZhmAyI7ZzH1BAytEp_bV/pubhtml?lsrp=1) (Google Sheet), [My Version](https://drive.google.com/file/d/1NrJKzMENRlp8Y-G4ATDq733imXs61b70/view?usp=sharing) (CSV)
* [RateMyProfessor UArizona Professors](https://www.ratemyprofessors.com/filter/professor/?&page=1&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid=1402) (JSON), [All pages merged to one](https://drive.google.com/file/d/1f8A51h7dCGqdNe4sZKekCi4aZOht23-n/view?usp=sharing) (JSON)
* [RateMyProfessor Reviews](https://www.ratemyprofessors.com/paginate/professors/ratings?tid=180399) (JSON) [All merged to one](https://drive.google.com/file/d/13QDkDkIZDkmzEon0eZYJ3QLVWlrDPtoQ/view?usp=sharing) (JSON)

The goals of this section are:

1. Extract the information from UArizona Salaries Sheet and load it into a csv file on my Google Drive, then load the Google Drive csv into a DataFrame
2. Extract the Professors from the RMP Professors API and load it into a json file on my Google Drive, then load it into a DataFrame
3. Find the Professors who exist in both datasets, and join these sets together into one DataFrame
4. Extract the reviews for each Professor in the merged DF from the RMP API, and load it into a json file on my Google Drive, then load it into a DataFrame

End Result:
* Have 2 DataFrames: merged_df and reviews_df

In [ ]:
# Extract Employees from UArizona Salaries Sheet

# get the salary file from Google Drive, name it salaries.csv
salaries_url = 'https://drive.google.com/file/d/1NrJKzMENRlp8Y-G4ATDq733imXs61b70/view?usp=sharing'
salaries_csv_drive = drive.CreateFile({'id':'1NrJKzMENRlp8Y-G4ATDq733imXs61b70'})
salaries_csv_drive.GetContentFile('salaries.csv')

# read salaries.csv into a df
salaries_df = pd.read_csv('salaries.csv')

# we need to transform the name column to FirstName and LastName so we can use it to get professor reviews from RMP

# split the Name column into FirstName and LastName
salaries_df[['LastName', 'FirstName']] = salaries_df['Name'].str.split(',', expand=True)

# remove middle names/initials from FirstName column
salaries_df['FirstName'] = salaries_df['FirstName'].str.split(expand=True)[0]

salaries_df

In [ ]:
# Extract the Professors from the RMP Professors API
# *** IMPORTANT: This cell shouldn't be run again because the results of this cell
#                 have been cached a Google Drive file. Rerunning will result in 
#                 unnecessary network traffic.
'''
rmp_professor_base = 'https://www.ratemyprofessors.com/filter/professor/?'
rmp_parameters = '&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid=1402'

rmp_url = rmp_professor_base + 'page=1' + rmp_parameters
# get the total number of UArizona Professors on RMP
with urllib.request.urlopen(rmp_url) as url:
    data = json.loads(url.read().decode())
    count_profs = data['searchResultsTotal']

# there are 20 professors listed on each page
pages_needed = math.ceil(count_profs / 20)

# get all of the professors from each page, save them in rmp_professors list
rmp_professors = []
for page_num in range(1, pages_needed + 1):
  rmp_url = rmp_professor_base + f'page={page_num}' + rmp_parameters
  with urllib.request.urlopen(rmp_url) as url:
    data = json.loads(url.read().decode())
    rmp_professors.append(data)

# now, merge each of these json pages into one json file object using the jsonmerge module

# this schema specifies to append each professor list to the already merged file
# instead of overwriting the previous one (which is the default)
schema = {
          "properties": {
              "professors": {
                  "mergeStrategy": "append"
              }             
          }
         }

# merge all of the json objects together, via a chaining method
merger = Merger(schema)
merged_professors = merger.merge(rmp_professors[0], rmp_professors[1])
for i in range(2, len(rmp_professors)):
  merged_professors = merger.merge(merged_professors, rmp_professors[i])

# dump the merged_professors json object into rmp_professors.json file
with open("rmp_professors.json", "w") as rmp_prof_json:
     json.dump(merged_professors, rmp_prof_json)

# I manually saved this dumped file to my Google Drive for future use
'''

In [ ]:
# Continue extracting the RMP Professors, now that we have the json files all merged into one

# load the json file from Google Drive
professors_url = 'https://drive.google.com/file/d/1f8A51h7dCGqdNe4sZKekCi4aZOht23-n/view?usp=sharing'
professors_json_drive = drive.CreateFile({'id':'1f8A51h7dCGqdNe4sZKekCi4aZOht23-n'})
professors_json_drive.GetContentFile('rmp_professors.json')
with open('rmp_professors.json', 'r') as rmp_json:
  rmp_professors = json.load(rmp_json)['professors']

# convert json to df
rmp_professors_df = pd.json_normalize(rmp_professors)

# rename first and last name columns so we can easily merge it with salaries_df

rmp_professors_df.rename(columns={'tFname' : 'FirstName', 'tLname' : 'LastName'}, inplace=True)

rmp_professors_df

In [ ]:
# Merge rmp_professors_df with salaries_df on FirstName and LastName
merged_df = pd.merge(rmp_professors_df, salaries_df, how='inner', on=['FirstName', 'LastName'])

merged_df

In [ ]:
# Now that we have the Professors who were found in both datasets, we can extract our last data set: reviews
# *** IMPORTANT: This cell shouldn't be run again because the results of this cell
#                 have been cached a Google Drive file. Rerunning will result in 
#                 unnecessary network traffic (took me about 4m 30s to send all of these requests)

'''
# get the first 20 reviews for each of the merged professors by using the RMP Reviews API
reviews_base_url = 'https://www.ratemyprofessors.com/paginate/professors/ratings?tid='
reviews = []
for i, tid in enumerate(merged_df['tid']):
  review_url = reviews_base_url + str(tid)
  with urllib.request.urlopen(review_url) as url:
    prof_reviews = json.loads(url.read().decode())

    # the retreived reviews don't include any information about the professor, so add the tid to all
    for review in prof_reviews['ratings']:
      review['tid'] = tid

    reviews.append(prof_reviews)

# this schema specifies to append each rating list to the already merged file
# instead of overwriting the previous one (which is the default)
schema = {
          "properties": {
              "ratings": {
                  "mergeStrategy": "append"
              }             
          }
         }

# merge all of the json objects together, via a chaining method
merger = Merger(schema)
merged_reviews = merger.merge(reviews[0], reviews[1])
for i in range(2, len(reviews)):
  merged_reviews = merger.merge(merged_reviews, reviews[i])

# dump the merged_professors json object into rmp_professors.json file
with open("rmp_reviews.json", "w") as rmp_reviews_json:
     json.dump(merged_reviews, rmp_reviews_json)

# I manually uploaded rmp_reviews.json to my Google Drive, so it can be used in the future
'''

In [ ]:
# get the reviews file from Google Drive, name it rmp_reviews.json
reviews_url = 'https://drive.google.com/file/d/13QDkDkIZDkmzEon0eZYJ3QLVWlrDPtoQ/view?usp=sharing'
reviews_json_drive = drive.CreateFile({'id':'13QDkDkIZDkmzEon0eZYJ3QLVWlrDPtoQ'})
reviews_json_drive.GetContentFile('rmp_reviews.json')

with open('rmp_reviews.json', 'r') as rmp_reviews_json:
  reviews_json = json.load(rmp_reviews_json)['ratings']

reviews_df = pd.json_normalize(reviews_json)

reviews_df

#Transform
Now we have all of the data loaded into two DataFrames: merged_df and reviews_df

We already did some transforming of the professor data to make our time easier to load the reviews data.

The goals of this section are:

1. Extract the useful columns and rename columns from reviews_df
2. Do the same for merged_df
3. Infer the genders and ethnicities for each professor
4. Create a department DataFrame by aggregating information from the merged_df
5. Create a course DataFrame by aggregating information from the reviews_df
6. Get rid of numpy types

End result:
* Have 4 DataFrames: merged_df, reviews_df, department_df and courses_df

In [ ]:
# make copies of the dfs
reviews_copy = reviews_df.copy()
merged_copy = merged_df.copy()

In [ ]:
# Transform reviews_df by dropping and renaming certain columns
reviews_df = reviews_copy.copy()

# drop the columns that aren't useful/are redundant
unnec_cols = ['unUsefulGrouping',	'usefulGrouping', 'teacher', 'sId', 'rTimestamp', 'rStatus', 'rErrorMsg', 'attendance', 'onlineClass', 
              'helpCount', 'notHelpCount', 'rTextBookUse', 'rWouldTakeAgain', 'takenForCredit', 'rInterest', 'rEasyString', 'rOverallString',
              'clarityColor', 'easyColor', 'helpColor', 'quality', 'teacherRatingTags']
reviews_df.drop(unnec_cols, axis='columns', inplace=True)

# rename the columns that are left to better names
reviews_df.rename(columns={'id':'ReviewID', 'rClarity':'ClarityRating', 'rClass':'CourseID', 'rComments':'Comment', 
                            'rDate':'ReviewDate', 'rEasy' : 'EasyRating', 'rHelpful':'HelpfulRating', 'rOverall':'OverallRating',
                            'teacherGrade':'CourseGrade', 'tid':'ProfID'}, inplace=True)

# convert CourseGrade to a numerical value
letter_num = {'A+':97,'A':93, 'A-':90, 'B+':87,'B':83, 'B-':80, 'C+':77,'C':73, 'C-':70, 'D+':67,'D':63, 'D-':60, 'F':50, 'E':50, 'WD':None, 'N/A':None,
              'Not sure yet':None, 'INC':None, 'Audit/No Grade':None, 'P':70}
reviews_df['CoursePercent'] = reviews_df.apply(lambda row: letter_num[row.CourseGrade], axis='columns')

reviews_df

In [ ]:
# Transform merged_df
merged_df = merged_copy.copy()

# drop unused or redundant columns in merged_df
unnec_cols = ['tSid', 'institution_name', 'contentType', 'categoryType', 'tMiddlename', 'Name', 'College Location', 
              'State Fund Ratio', 'FTE', 'College Name', 'rating_class', 'tDept']
merged_df.drop(unnec_cols, axis='columns', inplace=True)

# rename columns in merged_df
merged_df.rename(columns={'tid':'ProfID','tNumRatings':'NumRatings', 'overall_rating':'AverageRating', 
                          'College Name':'CollegeName', 'Primary Title':'Title', 
                          'Annual at Actual FTE':'ActualSalary', 'Annual at Full FTE':'FTESalary'}, inplace=True)

# remove the department name from the end of each professor's Title
merged_df['Title'] = merged_df['Title'].str.split(',', expand=True)[0]

# convert to numeric columns
merged_df.loc[merged_df['AverageRating'] == 'N/A', 'AverageRating'] = 0
merged_df['ActualSalary'] = merged_df['ActualSalary'].str.replace(',', '')
merged_df['FTESalary'] = merged_df['FTESalary'].str.replace(',', '')
merged_df = merged_df.astype({'NumRatings':'int64', 'AverageRating':'float64', 'ActualSalary':'int64', 'FTESalary':'int64'})

merged_copy = merged_df.copy()

merged_df

In [ ]:
# infer the gender for each professor
gender_detector = gender.Detector()
merged_df['InferredGender'] = merged_df.apply(lambda row: gender_detector.get_gender(row.FirstName), axis='columns')
# replace estimates with their full values, and unknown values with Nones
merged_df.loc[merged_df.InferredGender == 'mostly_male', 'InferredGender'] = 'male'
merged_df.loc[merged_df.InferredGender == 'mostly_female', 'InferredGender'] = 'female'
merged_df.loc[merged_df.InferredGender == 'unknown', 'InferredGender'] = None
merged_df.loc[merged_df.InferredGender == 'andy', 'InferredGender'] = None

print(f"{merged_df['InferredGender'].isnull().sum()/len(merged_df.index) * 100:.2f}% of professors are missing their gender")

merged_df

In [ ]:
# infer ethnicity for each professor
ethnicities = ['pctwhite', 'pctblack', 'pctapi', 'pctaian', 'pct2prace', 'pcthispanic']

# put the percent of each ethnicity for each professor into merged_df
merged_df = census_ln(merged_df, 'LastName', 2010)

# convert each pct column to a str because NaN values aren't being replaced using normal means (these are object typed fields)
merged_df = merged_df.astype({'pctwhite':'str', 'pctblack':'str', 'pctapi':'str', 'pctaian':'str', 'pct2prace':'str', 'pcthispanic':'str'})

# convert each invalid column value to a 0
for eth in ethnicities:
  merged_df.loc[merged_df[eth] == 'nan', eth] = 0
  merged_df.loc[merged_df[eth] == '(S)', eth] = 0

# convert the pct columns to floats now that we have removed NaNs and (S)s
merged_df = merged_df.astype({'pctwhite':'float64', 'pctblack':'float64', 'pctapi':'float64', 'pctaian':'float64', 'pct2prace':'float64', 'pcthispanic':'float64'})

# find the maximum confidence for each professor's ethnicity
merged_df['EthnicityConfidence'] = merged_df[['pctwhite', 'pctblack', 'pctapi', 'pctaian', 'pct2prace', 'pcthispanic']].max(axis='columns')

# assign the correct ethnicity to the professor based on the max confidence found
merged_df['InferredEthnicity'] = merged_df.apply(lambda row: row[row == row['EthnicityConfidence']].index[0], axis=1)

# remove the 'pct' from the beginning of the ethincities
for eth in ethnicities:
  merged_df.loc[merged_df['InferredEthnicity'] == eth, 'InferredEthnicity'] = eth[3:]

# replace all None ethnicities with nonwhite (new ethnicity) because I am assuming the data is pulling mostly from white surnames, 
# so most of the misses will be non-white
merged_df.loc[merged_df['EthnicityConfidence'] == 0, 'InferredEthnicity'] = 'nonwhite'

# drop the pct columns since they are no longer needed
merged_df.drop(ethnicities, axis='columns', inplace=True)

merged_df

In [ ]:
# create the department dataframe
department_df = pd.DataFrame({'Department':merged_df['Department'].unique()})

# get the number of professors in each department
aggs = pd.merge(department_df, merged_df, how='inner', on='Department').groupby('Department').agg(NumProfessors=('Department', 'count'), 
                                                                                                  AvgRating=('AverageRating', 'mean'),
                                                                                                  AvgFTESalary=('FTESalary', 'mean'),
                                                                                                  TotalProfPay=('ActualSalary', 'sum'))
department_df = pd.merge(department_df, aggs, how='inner', on='Department')
ethnicities = ['white', 'black', 'api', 'aian', '2prace', 'hispanic', 'nonwhite']
for eth in ethnicities:
  eth_aggs = pd.merge(department_df, merged_df[merged_df['InferredEthnicity'] == eth], 
                    how='inner', on='Department').groupby('Department').agg(
                        count=('InferredEthnicity', 'count')
                    )
  eth_aggs.columns = ['Count'+eth]
  department_df = pd.merge(department_df, eth_aggs, how='left', on='Department').fillna(0)

  department_df['Percent'+eth] = department_df.apply(lambda row: row['Count'+eth] / row['NumProfessors'], axis='columns')
  department_df.drop(['Count'+eth], axis='columns', inplace=True)

department_df

In [ ]:
# create courses df
courses_df = pd.DataFrame({'CourseID' : reviews_df['CourseID'].unique()})

courses_df = pd.merge(reviews_df, courses_df, how='inner', on='CourseID').groupby('CourseID').agg(AvgEasy=('EasyRating', 'mean'),
                                                                                            AvgHelpful=('HelpfulRating', 'mean'),
                                                                                            AvgClarity=('ClarityRating', 'mean'),
                                                                                            AvgOverall=('OverallRating', 'mean'),
                                                                                            MostCommonProf=('ProfID', lambda x: pd.Series.mode(x)[0]),
                                                                                            AvgGrade=('CoursePercent', 'mean'),
                                                                                            NumRatings=('ReviewID', 'count'))

# get the department of the most common professor for each course
courses_df.reset_index(level=0, inplace=True)
prof_dept = merged_df[['ProfID', 'Department']]
courses_df = pd.merge(courses_df, prof_dept, how='inner', left_on='MostCommonProf', right_on='ProfID')

courses_df

In [ ]:
# numpy types muck everything up
import datetime as dt
def numpy_to_python_types(df):
  for col in df.columns:
    t = df[col].dtype

    if t == 'int64' or t == 'float64':
      df = df.astype({col: 'object'})

  return df

merged_df = numpy_to_python_types(merged_df)

reviews_df = reviews_df.where(pd.notnull(reviews_df), None)
reviews_df = numpy_to_python_types(reviews_df)

department_df = numpy_to_python_types(department_df)

courses_df = courses_df.replace({np.nan: None})
courses_df = numpy_to_python_types(courses_df)

#Load
Now, we have merged_df (for professors), reviews_df, department_df, and courses_df.

So we need to create tables and insert all of the data into them.

End result:
* Have 4 tables: Professor, Review, Department, Course

In [ ]:
# SQL function definitions
def get_conn_cur(): # define function name and arguments (there aren't any)
  # Make a connection
  conn = mysql.connect(host="54.184.86.6", 
                       database="rmp_ista",
                       user="select_rmp",
                       password="select_rmp")

  cur = conn.cursor()   # Make a cursor after

  return(conn, cur)   # Return both the connection and the cursor

def run_query(query_string):
  conn, cur = get_conn_cur() # get connection and cursor

  cur.execute(query_string) # executing string as before

  my_data = cur.fetchall() # fetch query data as before

  # here we're extracting the 0th element for each item in cur.description
  colnames = [desc[0] for desc in cur.description]

  cur.close() # close
  conn.close() # close

  return pd.DataFrame(data=my_data, columns=colnames)
  #return(colnames, my_data) # return column names AND data

# make sql_head function
def sql_head(table_name):
  return run_query(f"SELECT * FROM {table_name} LIMIT 5")

# Check table_names
def get_table_names():
  conn, cur = get_conn_cur() # get connection and cursor

  # query to get table names
  table_name_query = """SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public' """

  cur.execute(table_name_query) # execute
  my_data = cur.fetchall() # fetch results

  cur.close() #close cursor
  conn.close() # close connection

  return(my_data) # return your fetched results

In [ ]:
# create the tables

conn, cur = get_conn_cur()

#cur.execute("DROP TABLE Professor")
#cur.execute("DROP TABLE Course")
#cur.execute("DROP TABLE Review")
#cur.execute("DROP TABLE Department")

# professor table
print(merged_df.columns)
create_professor = """CREATE TABLE Professor(
  FirstName VARCHAR(50),
  LastName VARCHAR(50),
  ProfID INT PRIMARY KEY,
  NumRatings INT,
  AverageRating REAL,
  Title VARCHAR(100),
  AcutalSalary INT,
  FTESalary INT,
  Department VARCHAR(100),
  InferredGender VARCHAR(10),
  EthnicityConfidence REAL,
  InferredEthnicity VARCHAR(10)
)"""

cur.execute(create_professor)

# create courses
print(courses_df.columns)
create_course = """CREATE TABLE Course(
  CourseID VARCHAR(20) PRIMARY KEY,
  AvgEasy REAL,
  AvgHelpful REAL,
  AvgClarity REAL,
  AvgOverall REAL,
  MostCommonProf INT,
  AvgGrade REAL,
  NumRatings INT,
  ProfID INT,
  Department VARCHAR(100)
)
"""
cur.execute(create_course)

# create reviews
print(reviews_df.columns)
create_review = """CREATE TABLE Review(
  ReviewID INT PRIMARY KEY,
  ClarityRating REAL,
  CourseID VARCHAR(20),
  Comment VARCHAR(5000),
  ReviewDate DATE,
  EasyRating REAL,
  HelpfulRating REAL,
  OverallRating REAL,
  CourseGrade VARCHAR(20),
  ProfID INT,
  CoursePercent REAL
)
"""
cur.execute(create_review)

# create departments
print(department_df.columns)
create_department = """CREATE TABLE Department(
  Department VARCHAR(100) PRIMARY KEY,
  NumProfessors INT,
  AvgRating REAL,
  AvgFTESalary REAL,
  TotalProfPay INT,
  Percentwhite REAL,
  Percentblack REAL,
  Percentapi REAL,
  Percentaian REAL,
  Percent2prace REAL,
  Percenthispanic REAL,
  Percentnonwhite REAL
)
"""

cur.execute(create_department)

cur.close()
conn.commit()
conn.close()

In [ ]:
# insert into professor table
conn, cur = get_conn_cur()

merged_df = merged_df.drop_duplicates('ProfID', keep='first')
#cur.execute("DELETE FROM Professor")

prof_tuples = merged_df.to_records(index=False)

for prof in prof_tuples:
  query = "INSERT INTO Professor VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

  cur.execute(query, tuple(prof))

cur.close()
conn.commit()
conn.close()


In [ ]:
sql_head('Professor')

In [ ]:
# insert into course table

conn, cur = get_conn_cur()

courses_df = courses_df.drop_duplicates('CourseID', keep='first')
#cur.execute("DELETE FROM Course")

course_tuples = courses_df.to_records(index=False)
for course in course_tuples:
  query = "INSERT INTO Course VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
  cur.execute(query, tuple(course))

cur.close()
conn.commit()
conn.close()


In [ ]:
sql_head('Course')

In [ ]:
# insert into department table

conn, cur = get_conn_cur()

department_df = department_df.drop_duplicates('Department', keep='first')
#cur.execute("DELETE FROM Department")

department_tuples = department_df.to_records(index=False)
for dept in department_tuples:
  query = "INSERT INTO Department VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
  cur.execute(query, tuple(dept))

cur.close()
conn.commit()
conn.close()


In [ ]:
sql_head('Department')

In [ ]:
# insert into review table

conn, cur = get_conn_cur()

reviews_df = reviews_df.drop_duplicates('ReviewID', keep='first')
#cur.execute("DELETE FROM Review")

review_tuples = reviews_df.to_records(index=False)
for rev in review_tuples:
  query = "INSERT INTO Review VALUES(%s, %s, %s, %s, STR_TO_DATE(%s, '%m/%d/%Y'), %s, %s, %s, %s, %s, %s)"

  cur.execute(query, tuple(rev))

cur.close()
conn.commit()
conn.close()


In [ ]:
sql_head("Review")

#Example Queries
Contains example queries based on some of the possible questions I outlined in HW7.

In [8]:
# average salary by gender
query = "SELECT InferredGender, AVG(FTESalary) AS AVGFTE, AVG(AcutalSalary) AS AVGActual FROM Professor GROUP BY InferredGender"

run_query(query)

,InferredGender,AVGFTE,AVGActual
0,male,102911.1816,97477.1517
1,female,88994.7819,83164.4977
2,None,94076.1326,88954.9724


In [9]:
# average salary by gender, grouped by department and title where title has at least two genders
depts_titles_2_genders = '''SELECT Department, Title
              FROM Professor 
              GROUP BY Department, Title
              HAVING COUNT(DISTINCT InferredGender) >= 2'''

other_restriction = "Department LIKE '%Math%'"

query = '''SELECT Department, Title, InferredGender, COUNT(InferredGender), AVG(FTESalary) AS AVGFTE, AVG(AcutalSalary) AS AVGActual 
            FROM Professor
            WHERE (Department, Title) IN (''' + depts_titles_2_genders + '''
            ) AND ''' + other_restriction + '''
            GROUP BY Department, Title, InferredGender
            ORDER BY Department, Title, InferredGender'''

run_query(query)

,Department,Title,InferredGender,COUNT(InferredGender),AVGFTE,AVGActual
0,Mathematics,Associate Professor,None,0,86964.2500,69764.2500
1,Mathematics,Associate Professor,female,2,84312.0000,84312.0000
2,Mathematics,Associate Professor,male,10,88238.3000,88238.3000
3,Mathematics,Instructor,None,0,42000.0000,42000.0000
4,Mathematics,Instructor,female,9,43444.4444,43444.4444
5,Mathematics,Instructor,male,8,42750.0000,42750.0000
6,Mathematics,Lecturer,None,0,48000.0000,48000.0000
7,Mathematics,Lecturer,female,2,48750.0000,42562.5000
8,Mathematics,Lecturer,male,1,48500.0000,48500.0000
9,Mathematics,Postdoctoral Research Associate I,None,0,47659.0000,47659.0000


In [10]:
# departments with more than 10 professors order by the highest minority ethnicity percent
query = """SELECT Department, NumProfessors, PercentWhite, 
            PercentBlack + PercentAPI + PercentAIAN + Percent2pRace + PercentHispanic + PercentNonWhite AS PercentMinority
            FROM Department
            WHERE NumProfessors > 10
            ORDER BY PercentMinority DESC"""
run_query(query)

,Department,NumProfessors,PercentWhite,PercentMinority
0,East Asian Studies,20,0.250000,0.750000
1,Finance,14,0.357143,0.642857
2,Systems and Industrial Engr,17,0.411765,0.588235
3,Spanish and Portuguese,34,0.411765,0.588235
4,Aerospace & Mechanical Engr,29,0.413793,0.586207
5,Africana Studies,13,0.461538,0.538462
6,Sch Middle E/N African Studies,13,0.461538,0.538462
7,Electrical and Computer Engr,29,0.482759,0.517241
8,Materials Science & Engr,12,0.500000,0.500000
9,Mathematics,97,0.525773,0.474227


In [11]:
# most to least difficult courses with more than 10 ratings
other_restriction = "Department LIKE '%Information%'"
query = """SELECT * FROM 
          Course
          WHERE NumRatings > 10 AND """ + other_restriction + """
          ORDER BY AvgEasy ASC"""
run_query(query)

,CourseID,AvgEasy,AvgHelpful,AvgClarity,AvgOverall,MostCommonProf,AvgGrade,NumRatings,ProfID,Department
0,ESOC150,2.153846,3.538462,3.461538,3.500000,1891506,93.222222,13,1891506,School of Information
1,MIS304,2.800000,4.040000,4.000000,4.020000,1705443,90.950000,25,1705443,Management Information Systems
2,MIS373,2.842105,4.105263,4.210526,4.157895,1786909,93.818182,19,1786909,Management Information Systems
3,MIS111,2.918919,3.162162,3.162162,3.162162,880998,89.000000,37,880998,Management Information Systems
4,MIS441,3.000000,1.437500,1.437500,1.437500,996649,90.000000,16,996649,Management Information Systems
5,ISTA130,3.250000,3.333333,3.333333,3.333333,980523,89.333333,12,980523,School of Information
6,ISTA116,3.476190,3.238095,3.095238,3.166667,2284722,85.166667,21,2284722,School of Information
7,MIS531,4.142857,2.357143,2.357143,2.357143,1667601,82.800000,14,1667601,Management Information Systems


In [12]:
# view all the ratings for professor named Jonathan Misurda
query = """SELECT Comment, OverallRating, CourseID, ReviewDate FROM 
          Review JOIN Professor ON (Review.ProfID=Professor.ProfID)
          WHERE FirstName='Jonathan' AND LastName = 'Misurda'"""

pd.set_option('display.max_colwidth', None)
run_query(query)

,Comment,OverallRating,CourseID,ReviewDate
0,One of the least caring professors I have ever encountered. He. Does. Not. Care. He is downright rude to students. I once mistakenly addressed him as Mr. instead of Dr. and he replied very rudely. AVOID!,1.0,CSC352,2019-12-29
1,He is funny but you can't deny his holier-than-thou attitude. He used to disregard me in office hours and participation because he genuinely enjoys humiliating a student who volunteers to contribute in class. Also I once asked him to review my bugged code and he scuffed at me &quot;your attempt isn't good enough for a rudimentary problem&quot; So opinionated,1.0,CSC352,2020-02-03
2,"The worest Teacher on this planet 100% sure of it. Can not get any worse. Doesnt help students, makes fun of students. He gives us projects to do and we have not even completed the lecture on what we need to complete the projects.",1.0,CSC352,2020-02-10
3,"Not the worst teacher in the world, at least compared to what I've heard about last semesters. But holy cow I don't understand the point of this class, everything he's gone over has already been taught in previous CS classes, so you just sit there while he over explains things that other teachers taught in four minutes that takes him an hour.",2.0,CSC335,2020-02-25
4,"Misurda has the style of teaching being, &quot;You're on your own&quot;. His lectures are very dry and hard to convey sometimes. For 5 weeks he did his 'slides' on a piece of paper where students would follow and copy notes. Come exam time, he doesn't have them. The class average for that exam was a 60 and our second midterm exam average was a 58. Dont recco",1.0,CSC352,2020-04-16
5,"Bruh, dark souls but no bonfires. Doesn't try to change the direction of the class to help the students but rather just to get on and get paid. Try to avoid at all cost.",1.0,CSC352,2020-05-06
6,"I really dont understand all the negative reviews for Dr. Misurda. His lectures are amazing and he is passionate about the subject material. If you dont wanna learn, dont take his course. If you do wanna learn, hes the professor for you. He goes very deep into the material. Outside of class his email response time is the quickest I've ever seen.",5.0,CSC335,2020-05-11
7,"Dr. Misurda gets a lot of hate, some of which I think is justifiable, but Dr. Misurda is definitely one of the best systems professors in CS. His 352 class is difficult, but you'll be thankful when you take OS (452). Once you've taken one of his exams, you know what to expect. Working hard in his classes will prepare you more than the alternatives.",5.0,CSC352,2020-05-12
8,"Garbage! Should be thrown out of teaching entirely. No piazza, no slides, nothing but his snide, long-winded lectures and demeaning jokes about any student that asks a question. I'm disgusted that I had to pay to take this guys class. He thinks he's god gift to man, constantly preening himself, but he never actually provides any value to my degree",1.0,CSC335CSC352,2020-05-15
9,"Teacher that does not like students despite what he himself thinks. Overall, the class was uncomfortable with an almost hostile feeling to it. He also does not understand modern approaches to software projects. Class also does not have a lot of resources. You will be punished for using internet for help (not code copying). Also no piazza.",1.0,CSC335,2020-09-23
